In [ ]:
#!pip install -q -r requirements.txt

In [ ]:
from fastai import *
from fastai.callbacks import *
from fastai.vision import *
import torchvision
from pretrainedmodels.models.resnext import resnext101_32x4d
from pretrainedmodels.models.senet import se_resnext50_32x4d

from metrics import *
from utils import *

In [ ]:
def _grayscale(x):
    if np.random.rand() < 0.5:
        i = torchvision.transforms.functional.to_pil_image(x)
        i = i.convert('L').convert('RGB')
        x = torchvision.transforms.functional.to_tensor(i)
    return x

grayscale = TfmPixel(_grayscale, order=20)

In [ ]:
input_dir = '../../whale'
output_dir = '.'
use_grayscale_images = False

In [ ]:
train_df = pd.read_csv(f'{input_dir}/train.csv')
train_df.describe()

In [ ]:
train_df = train_df[train_df.Id != 'new_whale']
train_df.describe()

In [ ]:
train_df = filter_samples_for_classes_with_few_occurrence(train_df, min_occurrences_to_include=1)
train_df.describe()

In [ ]:
xtra_tfms = [grayscale()] if use_grayscale_images else []
data = (
    ImageItemList
    .from_df(train_df, path=input_dir, folder='train')
    .split_by_idx(stratified_split_indexes(train_df, valid_pct=0.2, seed=42))
    .label_from_df(classes=sorted(train_df.Id.unique().tolist()))
    .transform(get_transforms(xtra_tfms=xtra_tfms), size=224, resize_method=ResizeMethod.SQUISH)
    .databunch(bs=64)
    #.normalize(imagenet_stats)
)

In [ ]:
print(f'dataset lengths: {len(data.train_ds)} / {len(data.valid_ds)}')

In [ ]:
train_unique_label_count = len({c.obj for c in data.train_ds.y})
valid_unique_label_count = len({c.obj for c in data.valid_ds.y})
print(f'unique label counts: {train_unique_label_count} / {valid_unique_label_count}')

In [ ]:
print(f'classes: {data.classes[:5]} ({len(data.classes)})')

In [ ]:
#data.show_batch(rows=3, ds_type=DatasetType.Train)

In [ ]:
learn = create_cnn(
    data,
    #models.resnet18,
    #lambda pretrained: resnext101_32x4d(pretrained='imagenet' if pretrained else None),
    lambda pretrained: se_resnext50_32x4d(pretrained='imagenet' if pretrained else None),
    pretrained=True,
    #split_on=resnet_split,
    cut=-3,
    lin_ftrs=[3096],
    metrics=[accuracy, map5],
    path=output_dir
)

In [ ]:
# learn.unfreeze()
# learn.lr_find()
# learn.recorder.plot()

In [ ]:
freeze_lr = 1e-2
unfreeze_lr = 1e-2

In [ ]:
learn.freeze()
learn.fit(5, lr=freeze_lr)
learn.save('model-stage-1')

In [ ]:
p, y = learn.get_preds(ds_type=DatasetType.Train)
print(f'accuracy: {accuracy(p, y):.6f}, map5: {map5(p, y):.6f}')

In [ ]:
learn.load('model-stage-1')
learn.unfreeze()
learn.fit(10, lr=slice(unfreeze_lr / 100, unfreeze_lr))
learn.save('model-stage-2')

In [ ]:
p, y = learn.get_preds(ds_type=DatasetType.Train)
print(f'accuracy: {accuracy(p, y):.6f}, map5: {map5(p, y):.6f}')

In [ ]:
learn.load('model-stage-2')
learn.unfreeze()
for _ in range(5):
    learn.fit_one_cycle(10, max_lr=unfreeze_lr)
    learn.save('model-stage-3')

In [ ]:
p, y = learn.get_preds(ds_type=DatasetType.Train)
print(f'accuracy: {accuracy(p, y):.6f}, map5: {map5(p, y):.6f}')